<a href="https://colab.research.google.com/github/mmnnf/Book_Store_app_with_filter/blob/main/vehicle/detection/vehicle_detection_93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
EfficientNet-B4 + GeM + ArcFace
‣ explicit train / valid / test klasörleri
"""

import os, random, numpy as np, torch, torch.nn as nn, torch.nn.functional as F
import torchvision.transforms as T, timm
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

# ────────────────────────────────────────────────
# 1) Katmanlar
# ────────────────────────────────────────────────
class GeM(nn.Module):
    def __init__(self, p=3.0, eps=1e-6):
        super().__init__(); self.p = nn.Parameter(torch.ones(1)*p); self.eps = eps
    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        return F.avg_pool2d(x, (x.size(-2), x.size(-1))).pow(1./self.p)[:, :, 0, 0]

try:
    from timm.layers import ArcMarginProduct
except ImportError:                   # timm<0.8 fallback
    class ArcMarginProduct(nn.Module):
        def __init__(self, in_f, out_f, s=30., m=0.40):
            super().__init__(); self.weight = nn.Parameter(torch.randn(out_f, in_f))
            self.s, self.m = s, m
        def forward(self, x, y):      # y kullanılmıyor (ArcFace yerine softmax)
            return self.s * F.linear(F.normalize(x), F.normalize(self.weight))

# ────────────────────────────────────────────────
# 2) Dataset
# ────────────────────────────────────────────────
class Cars(Dataset):
    def __init__(self, root, split, tf):
        self.root = os.path.join(root, split)
        self.tf   = tf
        self.cls  = sorted(os.listdir(self.root))
        self.c2i  = {c: i for i, c in enumerate(self.cls)}
        self.samples = [
            (os.path.join(self.root, c, f), self.c2i[c])
            for c in self.cls
            for f in os.listdir(os.path.join(self.root, c))
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        p, y = self.samples[idx]
        img  = Image.open(p).convert('RGB')
        return self.tf(img), y
    def classes(self): return self.cls

# ────────────────────────────────────────────────
# 3) Ana akış
# ────────────────────────────────────────────────
def main():
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"PyTorch {torch.__version__} | CUDA {torch.version.cuda or '-'}")

    ROOT  = r"C:\Users\User\PycharmProjects\Masin_model_detect\balanv\RoboFlow_Cars2\cars_dataset_by_brand_3split"
    CKPT  = "effb4_val70plus_ROBOFLOW_brend_BALANCED_augmented.pth"
    BATCH = 4
    EPOCHS, LR = 60, 3e-4
    torch.manual_seed(42); random.seed(42); np.random.seed(42)

    # ── Dönüşümler
    MEAN, STD = [0.485,0.456,0.406], [0.229,0.224,0.225]
    train_tf = T.Compose([
        T.RandomResizedCrop(380, scale=(0.7,1.0), ratio=(0.75,1.33)),
        T.RandomHorizontalFlip(), T.ColorJitter(0.3,0.3,0.3,0.1),
        T.RandomPerspective(0.2), T.ToTensor(), T.Normalize(MEAN, STD)
    ])
    val_tf = T.Compose([
        T.Resize(400), T.CenterCrop(380),
        T.ToTensor(), T.Normalize(MEAN, STD)
    ])

    # ── Kümeleri oku (artık hazır klasörler)
    train_ds = Cars(ROOT, 'train',  train_tf)
    val_ds   = Cars(ROOT, 'valid',  val_tf)
    test_ds  = Cars(ROOT, 'test',   val_tf)

    n_cls = len(train_ds.classes())
    print("Sınıf sayısı :", n_cls, "| Train örnek:", len(train_ds),
          "| Valid:", len(val_ds), "| Test:", len(test_ds))

    # ── DataLoader’lar
    dl_kw = dict(batch_size=BATCH, num_workers=2, pin_memory=True)
    train_dl = DataLoader(train_ds, shuffle=True,  **dl_kw)
    val_dl   = DataLoader(val_ds,   shuffle=False, **dl_kw)
    test_dl  = DataLoader(test_ds,  shuffle=False, **dl_kw)

    # ────────────────────────────────────────────
    # 4) Model
    # ────────────────────────────────────────────
    backbone = timm.create_model('efficientnet_b4', pretrained=True,
                                 num_classes=0).to(DEVICE)
    backbone.global_pool = GeM().to(DEVICE)
    head = ArcMarginProduct(backbone.num_features, n_cls, s=30., m=0.40).to(DEVICE)

    crit  = nn.CrossEntropyLoss(label_smoothing=0.1)
    opt   = torch.optim.AdamW(
        list(backbone.parameters())+list(head.parameters()),
        lr=LR, weight_decay=1e-2)
    sched  = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)
    scaler = torch.cuda.amp.GradScaler(enabled=DEVICE.type=='cuda')

    # ── Döngü
    def run(loader, train=True):
        backbone.train(train); head.train(train)
        tot = loss_sum = corr = 0
        for step, (x,y) in enumerate(tqdm(loader, leave=False)):
            x,y = x.to(DEVICE), y.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=DEVICE.type=='cuda'):
                feats  = backbone(x); logits = head(feats, y)
                loss   = crit(logits, y)
            if train:
                opt.zero_grad(); scaler.scale(loss).backward()
                scaler.step(opt); scaler.update()
            loss_sum += loss.item()*x.size(0)
            corr     += (logits.argmax(1)==y).sum().item()
            tot      += x.size(0)
        return loss_sum/tot, 100.*corr/tot

    best_val = 0.
    for ep in range(1, EPOCHS+1):
        print(f"\n🟢 Epoch {ep:02d}/{EPOCHS}")
        tl, ta = run(train_dl, True)
        vl, va = run(val_dl,   False)
        sched.step()

        print(f"E{ep:02d} | tr {ta:5.1f}% | val {va:5.1f}%")
        if va>=70 and va>best_val:
            best_val = va
            torch.save({'backbone': backbone.state_dict(),
                        'head': head.state_dict(),
                        'val_acc': va, 'epoch': ep}, CKPT)
            print(f" 💾  {va:4.1f}% ≥70 → kaydedildi ({CKPT})")

    # ── En iyi modeli yükle & test
    if os.path.exists(CKPT):
        ckpt = torch.load(CKPT, map_location=DEVICE)
        backbone.load_state_dict(ckpt['backbone'])
        head.load_state_dict(ckpt['head'])
        print(f"\nEn iyi model: epoch {ckpt['epoch']} | val {ckpt['val_acc']:.1f}%")

    _, test_acc = run(test_dl, False)
    print(f"\nTest top-1 : {test_acc:.2f}%")

# ────────────────────────────────────────────────
if __name__ == "__main__":
    import torch.multiprocessing as mp
    mp.set_start_method("spawn", force=True)
    main()
